In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import surprise
from sklearn.model_selection import train_test_split#, cross_validate
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans, SVD, SVDpp, SlopeOne
from surprise import accuracy
from sklearn.preprocessing import normalize
import scipy
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from sklearn.metrics import mean_squared_error

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [11]:
books = pd.read_csv('goodbooks-10k/books.csv')
ratings = pd.read_csv('goodbooks-10k/ratings.csv')
toread = pd.read_csv('goodbooks-10k/to_read.csv')
tags = pd.read_csv('goodbooks-10k/tags.csv')
book_tags = pd.read_csv('goodbooks-10k/book_tags.csv')


# pint(books.shape, '\n', books.head())
# print(ratings.shape, '\n', ratings.head())
# print(toread.shape, '\n', toread.head())
# print(tags.shape, '\n', tags.tail())
# print(book_tags.shape , '\n',  book_tags.head())

# len(ratings['user_id'].unique()), len(ratings['book_id'].unique())
# ratings[ratings['user_id']==50]

# print('Before : ', ratings.shape[0])
# ratings.drop_duplicates(inplace=True)
# print('After dropping duplicates: ', ratings.shape[0])
# print('The number of unique users we have is:', len(ratings.user_id.unique()))
# print('The number of unique books we have is:', len(ratings.book_id.unique()))
# print("The median user rated %d books."%ratings.user_id.value_counts().median())
# print('The max rating is: %d,'%ratings.rating.max()," and the min rating is: %d"%ratings.rating.min())

## Recommend books based on authors

##### Concatenate (first name + last name) of author

In [12]:
# # Merge 'tags' and 'book_tags' 
# tags_joined = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')

# # item_ids
# item_ids = books['book_id'].tolist()

# # author to feature

# tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# tfidf_matrix = tf.fit_transform(books['authors'].head(10000))
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# # Build a 1-d array with book titles

# titles = books['title']
# indices = pd.Series(books.index, index=books['title'])

# # Get book recommendation based on cosine similarity
# def authors_recommendations(title):
#     idx = indices[title]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
#     sim_scores = sim_scores[1:21]
#     book_indices = [i[0] for i in sim_scores]
#     return titles.iloc[book_indices]

# result = authors_recommendations('The Hobbit')

## Prepare Data

In [13]:
dataset = ratings[['user_id','book_id','rating']]
dataset.columns = ['user','item','rating']
dataset = dataset[0:100000]

train_split, test_split = train_test_split(dataset, test_size=0.25)

reader = surprise.Reader(rating_scale=(1,5))
train_data = surprise.Dataset.load_from_df(train_split, reader)
test_data = surprise.Dataset.load_from_df(test_split, reader)

rmse_knn = []
rmse_cb = []

# collab_knn.get_neighbors(50, 5)    # .get_neighbors() returns closest items to given arg

## Build user profile

In [14]:
def get_item_profile(item_id, item_ids, tfidf_matrix):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile


def get_item_profiles(ids, item_ids, tfidf_matrix):
    item_profiles_list = [get_item_profile(x, item_ids, tfidf_matrix) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles


def build_users_profile(user_id, train_indexed, item_ids, tfidf_matrix):
    train_user_df = train_indexed.loc[user_id]
    user_item_profiles = get_item_profiles(pd.Series(train_user_df['item']), item_ids, tfidf_matrix)
    user_item_strengths = np.array(train_user_df['rating']).reshape(-1,1)
    
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = normalize(user_item_strengths_weighted_avg)
    return user_profile_norm


def build_users_profiles(item_ids, tfidf_matrix): 
    train_indexed = train_split[train_split['item'].isin(books.index+1)].set_index('user')
    user_profiles = {}
    for user_id in train_indexed.index.unique():
        user_profiles[user_id] = build_users_profile(user_id, train_indexed, item_ids, tfidf_matrix)
        
    return user_profiles

## Recommend books based on tags

In [15]:
class ContentBasedRecommender():
    
    def __init__(self, books, book_tags, tags):
        self.model_name = 'Content-Based-Recommender'
        self.books = books
        self.titles = books['title']
        self.indices = pd.Series(books.index, index=books['title'])
        self.tags_joined = pd.merge(book_tags, tags, 
                                    left_on='tag_id', right_on='tag_id', how='inner')
        self.books_with_tags = pd.merge(books, self.tags_joined, 
                                        left_on='book_id', right_on='goodreads_book_id', how='inner')
        self.item_ids  = self.get_item_ids()
        self.tfidf_matrix = self.get_tfidf()
        self.user_profiles = build_users_profiles(self.item_ids, self.tfidf_matrix)

    
    def get_item_ids(self):
        self.item_ids = (self.books['id']).tolist()
        return self.item_ids
    
    
    def get_tfidf(self):
        tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 2),
                             min_df=0,
                             max_features=1000,
                             stop_words='english')
        tfidf_matrix = tf.fit_transform(self.books_with_tags['tag_name'].head(10000))
        return tfidf_matrix
    
    
    def get_cosine_similarity(self, tfidf_matrix1, tfidf_matrix2):
        print(tfidf_matrix1.shape)  #user_profile
        print(tfidf_matrix2.shape)  #base tfidf
        cosine_sim = linear_kernel(tfidf_matrix1, tfidf_matrix2)
        
        return cosine_sim
    
    
    def _get_similar_items_to_user_profile(self, user_id, topn=1000):
        cosine_similarities = self.get_cosine_similarity(self.user_profiles[user_id], self.tfidf_matrix)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_items = sorted([(self.item_ids[i], cosine_similarities[0,i]) for i in similar_indices], 
                               key=lambda x: -x[1])
        return similar_items
    
    
    def get_recommendations(self, user_id, topn=1000):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        #Ignores items the user has already interacted --- LATER
        
        recommendations_df = pd.DataFrame(similar_items, columns=['book_id', 'rating']).head(topn)
        recommendations_df = recommendations_df.merge(self.books, how = 'left', 
                                                          left_on = 'book_id', 
                                                          right_on = 'id')[['id', 'original_title', 'rating']]
        return recommendations_df


In [16]:
# cb_model = ContentBasedRecommender(books, book_tags, tags)

# result = cb_model.get_recommendations(29703)
# result

## Cross validating collaborative filtering algorithms

In [18]:
benchmark = []

# Iterate over all algorithms
# for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:

for algorithm in [SVD(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    print(algorithm)
    results = cross_validate(algorithm, train_data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

# Best perforing algorithms
# SVD, BaselineOnly, KNNBaseline, KNNWithMeans, KNNWithZScore

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.911299,6.615524,0.468815
BaselineOnly,0.912511,0.231861,0.325903
KNNBaseline,0.920036,1.672253,6.355297
KNNWithMeans,0.928655,2.189351,6.003163
KNNWithZScore,0.929752,2.150114,6.427357
CoClustering,0.942115,4.393441,0.521624
SlopeOne,0.951473,0.361955,1.812613
NMF,0.978210,7.342017,0.489212
KNNBasic,0.990543,1.406821,5.508454


### Make recommendations for a user

In [19]:
class Recommender():
    
    def __init__(self, dataset, books, train_data, test_data, algorithm):
        self.algorithm = algorithm
        self.dataset = dataset
        self.trainset = train_data.build_full_trainset()
        self.train_data = train_data
        self.testset = test_data.build_full_trainset().build_testset()
        self.test_data = test_data
        self.books = books
    
    
    def get_unrated_items(self, user_id):
        items = self.dataset['item'].unique()
        items_U = self.dataset.loc[self.dataset['user'] == user_id, 'item']
        items_to_pred = np.setdiff1d(items, items_U)
        return items_to_pred
    
    
    def rate_unrated_items(self, user_id, items_to_pred):
        # build testset for user
        testset_U = pd.DataFrame([[user_id, item, 4.0] for item in items_to_pred])  
        testset_U = surprise.Dataset.load_from_df(testset_U, reader) 
        testset_U = testset_U.build_full_trainset().build_testset()
        recommendations = self.algorithm.test(testset_U)

        return recommendations
    
    
    def get_Iu(self, uid):
        
        trainset = self.algorithm.trainset
        try:
            return len(trainset.ur[trainset.to_inner_uid(uid)])
        except ValueError: # user was not part of the trainset
            return 0
    
    
    def get_Ui(self, iid):
        
        trainset = self.algorithm.trainset
        try: 
            return len(trainset.ir[trainset.to_inner_iid(iid)])
        except ValueError:
            return 0
    
    
    def get_recommendations_for_user(self, user_id):
        
        items_to_pred = self.get_unrated_items(user_id)
        recommendations = self.rate_unrated_items(user_id, items_to_pred)
        
        df = pd.DataFrame(recommendations, columns=['uid', 'iid', 'rui', 'est', 'details'])
        df['err'] = abs(df.est - df.rui)
        
        recommendations_df = df.merge(self.books, how = 'left', 
                                                  left_on = 'iid', 
                                                  right_on = 'id')[['iid', 'original_title', 'est']]
        recommendations_df = recommendations_df.sort_values(['est'], ascending=False)
        
        return recommendations_df

In [ ]:
# sim_options = {'name': 'cosine',
#                'user_based': False}
# collab_knn = surprise.KNNBasic(k=40,sim_options=sim_options)
# trainset = train_data.build_full_trainset()
# collab_knn.fit(trainset)
# testset = test_data.build_full_trainset().build_testset()
# preds = collab_knn.test(testset)
# surprise.accuracy.rmse(preds, verbose=True)
# # recommender = Recommender(dataset, books, train_data, test_data, collab_knn)
# # recommender.get_recommendations_for_user(29703)
# # collab_knn.fit()
# # collab_knn.get_recommendations_for_user(29703)

In [ ]:
# rmse_knn = []
# sim_options = {'name': 'cosine',
#                'user_based': False}
# collab_knn = surprise.KNNBasic(k=40,sim_options=sim_options)

# kSplit = surprise.model_selection.split.KFold(n_splits=5, shuffle=True)

# for trainset, testset in kSplit.split(train_data): #iterate through the folds.
#     collab_knn.fit(trainset)
#     preds_knn = collab_knn.test(testset)
#     rmse_knn.append(surprise.accuracy.rmse(preds_knn,verbose=True))

## Ensembling
Now, we ensemble the two recommenders above:
1. SVD
2. BaselineOnly
3. KNNBaseline
4. KNNWithMeans
5. KNNWithZScore

In [64]:
class HybridRecommender():
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, rs_list, wt_list, train_data, test_data):
        
        self.rs = rs_list 
        self.weights = wt_list
        self.num_algo = len(rs_list)
        self.trainset = train_data.build_full_trainset()
        self.testset = test_data.build_full_trainset().build_testset()
    
    def test(self):
        
        df = []
        rmse_rs = []
        for i in range(self.num_algo):
            self.rs[i].fit(self.trainset)
            preds_rs = self.rs[i].test(self.testset)
            rmse_rs.append(surprise.accuracy.rmse(preds_rs, verbose=True))
            df.append(pd.DataFrame(preds_rs, columns=['uid', 'iid', 'rui', 'est', 'details']))
            
        test_df = pd.DataFrame(self.testset, columns=['user', 'item', 'rating'])

        preds_net = 0
        for i in range(self.num_algo):
            preds_net += self.weights[i]*df[i]['est']
        preds_net /= sum(self.weights)
        
        rmse_net = mean_squared_error(test_df['rating'], preds_net, squared=False)
   
        return rmse_rs, rmse_net
        
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        
        #Ignores items the user has already interacted --- LATER  
                
        # get the top-1000 cb_model recommendations
        cb_recs = self.cb_model.get_recommendations(user_id)
        
        # get the top-1000 collab_knn recommendation
        knn_recs = self.collab_knn.get_recommendations_for_user(user_id)
        
        # combine results by id
        recomm = cb_recs.merge(knn_recs,
                                   how = 'outer', 
                                   left_on = 'id', 
                                   right_on = 'iid').fillna(0.0)
        
        # compute a HYBRID recommendation score based on knn and svd
        recomm['score_hybrid'] = (recomm['rating'] * self.cb_ensemble_weight) + (recomm['est'] * self.knn_ensemble_weight)
                                    
        
        recommendations = recomm.sort_values('score_hybrid', ascending=False).head(topn)
        
        return recommendations

In [85]:
seed = 6
svd = SVD(random_state=seed)
baseline = BaselineOnly()
knn_baseline = KNNBaseline(random_state=seed)
knn_means = KNNWithMeans(random_state=seed)
knn_zscore = KNNWithZScore(random_state=seed)
cocluster = CoClustering(random_state=seed)

rs_list = [svd, baseline, knn_baseline, knn_means, knn_zscore]
wt_list = [0.7, 0.7, 1, 0.5, 0.5]

hybrid_rs = HybridRecommender(rs_list, wt_list, train_data, test_data)
r, rn = hybrid_rs.test()

RMSE: 0.8902
Estimating biases using als...
RMSE: 0.8936
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8840
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9005
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9002


In [86]:
print('RMSE list : ', r)
print('RMSE net : ', rn)

RMSE list :  [0.8902136543637234, 0.893594918712878, 0.8839550444010758, 0.9005142246712313, 0.900156408865128]
RMSE net :  0.8778993451319257


In [63]:
sum(wt_list)

4.3999999999999995